In [18]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# SparkSession 생성
spark = SparkSession.builder \
    .appName("DeviceIoTData") \
    .getOrCreate()

from dataclasses import dataclass

class DeviceIoTData:
    battery_level: int
    c02_level: int
    cca2: str
    cca3: str
    cn: str
    device_id: int
    device_name: str
    humidity: int
    ip: str
    latitude: float
    lcd: str
    longitude: float
    scale: str
    temp: int
    timestamp: int

df = spark.read.json("/Users/sunghoon/Desktop/file/iot_devices-2.json") #본인 파일 경로


df.show(5, truncate=False)

filterTempDF = df.filter((df.temp > 30) & (df.humidity > 70))

filterTempDF = filterTempDF.drop('scale', 'timestamp') #혹시나 겹치는 부분이 있다면 제거 (안해도 됨)

filterTempDF.show(5, truncate = False)

from pyspark.sql.functions import col #지정을 위해서 Col 가져옴

class DeviceTempByCountry:
    temp: int
    device_name: str
    device_id:int
    cca3: str
# temp가 25 이상인 데이터 필터링
dfTemp = df.filter(col('temp') > 25) \
           .select(
               col('temp'),
               col('device_name'),
               col('device_id'),
               col('cca3')
           )

# 결과 확인
dfTemp.show(5, truncate=False)

#데이터 세트 첫 열 

device = dfTemp.columns[0]

device_c = df.select(device)

device_1.show(5)

#데이터 세트 첫 행

device_r = dfTemp.first()

print(device_r)

#where 을 활용한 방법

dfTemp2 = (df.select(col("temp"), col("device_name"), col("device_id"), col("cca3"))
             .where(df.temp > 25 ))

dfTemp2.show(5, truncate = False)







+-------------+---------+----+----+-------------+---------+---------------------+--------+-------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|cn           |device_id|device_name          |humidity|ip           |latitude|lcd   |longitude|scale  |temp|timestamp    |
+-------------+---------+----+----+-------------+---------+---------------------+--------+-------------+--------+------+---------+-------+----+-------------+
|8            |868      |US  |USA |United States|1        |meter-gauge-1xbYRYcj |51      |68.161.225.1 |38.0    |green |-97.0    |Celsius|34  |1458444054093|
|7            |1473     |NO  |NOR |Norway       |2        |sensor-pad-2n2Pea    |70      |213.161.254.1|62.47   |red   |6.15     |Celsius|11  |1458444054119|
|2            |1556     |IT  |ITA |Italy        |3        |device-mac-36TWSKiT  |44      |88.36.5.1    |42.83   |red   |12.83    |Celsius|19  |1458444054120|
|6            |1080     |US  |USA |United States|4  